<a href="https://colab.research.google.com/github/tanyagupta1/Machine-Learning/blob/main/NLP/sentiment_embedding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np
import pandas as pd

In [ ]:
!wget --no-check-certificate \
    -O /tmp/sentiment.csv https://drive.google.com/uc?id=13ySLC_ue6Umt9RJYSeM2t-V0kCv-4C-P


In [4]:
dataset = pd.read_csv('/tmp/sentiment.csv')
sentences = dataset['text'].tolist()
labels = dataset['sentiment'].tolist()

In [5]:
training_size = int(len(sentences) * 0.8)

training_sentences = sentences[0:training_size]
testing_sentences = sentences[training_size:]
training_labels = labels[0:training_size]
testing_labels = labels[training_size:]

training_labels_final = np.array(training_labels)
testing_labels_final = np.array(testing_labels)

In [7]:
tokenizer=Tokenizer(num_words=1000,oov_token="<OOV>")
tokenizer.fit_on_texts(training_sentences)
sequences=tokenizer.texts_to_sequences(training_sentences)

In [9]:
padded=pad_sequences(sequences,maxlen=100,padding='post',truncating='post')
testing_seq=tokenizer.texts_to_sequences(testing_sentences)
padded_test=pad_sequences(testing_seq,maxlen=100,padding='post',truncating='post')

In [21]:
tokenizer.word_index
reverse_word_index=dict([(value,key) for (key,value) in tokenizer.word_index.items()])
def decode(text):
  return ' '.join([reverse_word_index.get(i,'?') for i in text])
print(decode(padded[1]))
print(training_sentences[1])

good case excellent value ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ?
Good case Excellent value.


In [24]:
model=tf.keras.Sequential([
                           tf.keras.layers.Embedding(1000,16,input_length=100),
                           tf.keras.layers.Flatten(),
                           tf.keras.layers.Dense(6,activation='relu'),
                           tf.keras.layers.Dense(1,activation='sigmoid')
])
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [25]:
model.fit(padded,training_labels_final,epochs=10,validation_data=(padded_test,testing_labels_final))

Epoch 1/10
50/50 [==============================] - 4s 9ms/step - loss: 0.6931 - accuracy: 0.5173 - val_loss: 0.6936 - val_accuracy: 0.4311
Epoch 2/10
50/50 [==============================] - 0s 5ms/step - loss: 0.6919 - accuracy: 0.5449 - val_loss: 0.6943 - val_accuracy: 0.4311
Epoch 3/10
50/50 [==============================] - 0s 5ms/step - loss: 0.6880 - accuracy: 0.6861 - val_loss: 0.6892 - val_accuracy: 0.6516
Epoch 4/10
50/50 [==============================] - 0s 6ms/step - loss: 0.6748 - accuracy: 0.7671 - val_loss: 0.6723 - val_accuracy: 0.6667
Epoch 5/10
50/50 [==============================] - 0s 6ms/step - loss: 0.6439 - accuracy: 0.7382 - val_loss: 0.6425 - val_accuracy: 0.6867
Epoch 6/10
50/50 [==============================] - 0s 6ms/step - loss: 0.5806 - accuracy: 0.8035 - val_loss: 0.5930 - val_accuracy: 0.7393
Epoch 7/10
50/50 [==============================] - 0s 5ms/step - loss: 0.4321 - accuracy: 0.9058 - val_loss: 0.5437 - val_accuracy: 0.7444
Epoch 8/10
50/50 [==

In [26]:
fake_reviews = ['I love this phone', 'I hate spaghetti', 
                'Everything was cold',
                'Everything was hot exactly as I wanted', 
                'Everything was green', 
                'the host seated us immediately',
                'they gave us free chocolate cake', 
                'not sure about the wilted flowers on the table',
                'only works when I stand on tippy toes', 
                'does not work when I stand on my head']
sample_sequence=tokenizer.texts_to_sequences(fake_reviews)
padded_fake=pad_sequences(sample_sequence,padding='post',maxlen=100)
classes=model.predict(padded_fake)
print(classes)

[[0.9638666 ]
 [0.10184738]
 [0.45241785]
 [0.60820466]
 [0.5135994 ]
 [0.74460083]
 [0.7685561 ]
 [0.05052236]
 [0.8866606 ]
 [0.00199402]]
